In [6]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

CLIENT_ID = os.getenv("IGDB_CLIENT_ID")
CLIENT_SECRET = os.getenv("IGDB_CLIENT_SECRET")

TOKEN_URL = "https://id.twitch.tv/oauth2/token"
IGDB_BASE_URL = "https://api.igdb.com/v4"

In [7]:
def get_igdb_access_token():
    resp = requests.post(
        TOKEN_URL,
        params={
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "grant_type": "client_credentials"
        }
    )
    resp.raise_for_status()
    return resp.json()["access_token"]

In [8]:
def fetch_game_story(game_name, limit=1):
    token = get_igdb_access_token()

    headers = {
        "Client-ID": CLIENT_ID,
        "Authorization": f"Bearer {token}"
    }

    query = f"""
    search "{game_name}";
    fields name, summary, storyline, genres.name, themes.name;
    limit {limit};
    """

    resp = requests.post(
        f"{IGDB_BASE_URL}/games",
        headers=headers,
        data=query
    )
    resp.raise_for_status()

    return resp.json()

In [9]:
def build_game_story_doc(game_name):
    games = fetch_game_story(game_name)

    if not games:
        raise ValueError("게임을 찾을 수 없습니다.")

    g = games[0]

    return {
        "title": g.get("name"),
        "summary": g.get("summary"),
        "storyline": g.get("storyline"),
        "genres": [x["name"] for x in g.get("genres", [])],
        "themes": [x["name"] for x in g.get("themes", [])]
    }

In [11]:
doc = build_game_story_doc("Rabbit and Steel")

print("🎮", doc["title"])
print("\n📖 Summary\n", doc["summary"])
print("\n📜 Storyline\n", doc["storyline"])

🎮 Rabbit & Steel

📖 Summary
 Rabbit & Steel is a co-op roguelike where rabbits must team up together in raids to investigate strange happenings in the Moonlit Kingdom.

📜 Storyline
 A few months ago, all contact with the Moonlit Kingdom was lost.
Reports indicate that the city's denizens attack outsiders on sight.
Investigative expeditions are repelled or go missing.
A colossal white tower now manifests in the city's center each night, stretching skyward towards the stars.
Something must be done. The rabbits will take care of this.
The night is short. Move swiftly.


In [16]:
from langchain_core.prompts import ChatPromptTemplate

MOOD_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     """You are a professional game narrative analyst.
Your task is to analyze a game's storyline and infer the game's overall mood and atmosphere.

You MUST return a valid JSON object only.
Do NOT include explanations or extra text.

Definitions:
- overall_mood: one of [dark, lighthearted, comedic, serious, neutral, epic]
- tones: descriptive tags such as cute, whimsical, tragic, cooperative, horror, fantasy, heroic
- emotional_axes: float values between 0.0 and 1.0
    - darkness
    - humor
    - violence
    - wholesomeness
    - seriousness
"""),
    ("human",
     """Storyline:
{storyline}

Return JSON in this exact format:
{{
  "overall_mood": "",
  "tones": [],
  "emotional_axes": {{
    "darkness": 0.0,
    "humor": 0.0,
    "violence": 0.0,
    "wholesomeness": 0.0,
    "seriousness": 0.0
  }},
  "one_line_description": ""
}}
""")
])

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

mood_chain = MOOD_PROMPT | llm | JsonOutputParser()

In [14]:
def analyze_game_mood_with_llm(storyline: str):
    return mood_chain.invoke({
        "storyline": storyline
    })

In [ ]:
doc = build_game_story_doc("Slay the Spire")
if doc["storyline"]:
    mood = analyze_game_mood_with_llm(doc["storyline"])
    print("🎭 Overall Mood:", mood["overall_mood"])
    print("🎨 Tones:", mood["tones"])
    print("📊 Emotional Axes:", mood["emotional_axes"])
    print("📝 One-liner:", mood["one_line_description"])
else:
    print("No storyline available to analyze.")

None
No storyline available to analyze.
